# Setup & Training Script
To be run after new videos are added

Based on the article [Guide to Build Video Classification Model](https://www.analyticsvidhya.com/blog/2019/09/step-by-step-deep-learning-tutorial-video-classification-python/)


In [1]:
!pip3 install opencv-python

### Importing Libraries

In [2]:
import os, os.path
import cv2  # for caputring videos
import math # for mathematical operations
import matplotlib.pyplot as plt # for plotting the images
%matplotlib inline
import pandas as pd
import numpy as np # for mathematical operations
from keras.utils import np_utils
from numpy import genfromtxt
from skimage.transform import resize # for resizing images
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm

import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.applications.vgg16 import preprocess_input

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from skimage.transform import resize 
from glob import glob
import os, os.path

Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

### Storing the name of videos in a dataframe
First bit opens the file, reads the file, and splits them by "enter"

Second bit creates a dataframe and stores the above extracted video names under "train['video_name']"

In [3]:
# open the .txt file which have names of training videos
# f = open("trainlist_tennis.txt", "r")
# temp = f.read()
# videos = temp.split("\n")
# print(videos)

# creating a dataframe having video names
# train = pd.DataFrame()
# train['video_name'] = videos
# train

 # Turn the csv files into dictionaries
1. Opens and converts csv file
2. Gets tags
3. Gets points for each frame
4. Reshapes each frame array
5. Groups frames by three

In [4]:
path="dataPoints_training/"

train = pd.DataFrame()

for filename in glob(os.path.join(path, '*.csv')):    
    file = filename.split("/")[1]
    train = pd.read_csv(path + file, header=None)
    locals()['trainingFrames{}'.format("_" + file.split(".csv")[0])] = {}
    locals()['trainingCombo{}'.format("_" + file.split(".csv")[0])] = {}
    
    tag = filename.split("/")[1].split("_")[3].split(".csv")[0]
    locals()['trainingCombo{}'.format("_" + file.split(".csv")[0])]["tag"] = tag            
        
    size = len(train[0]) // 12
       
    n = 12
    for i in range((len(train) + 12 - 1) // 12):
        data = train.to_numpy()[i * n:(i + 1) * n]
        data = np.delete(data, 0, 1)
        data = np.reshape(data, (24, 1))
        locals()['trainingFrames{}'.format("_" + file.split(".csv")[0])]['{}'.format(i)] = data

    for i in locals()['trainingFrames{}'.format("_" + file.split(".csv")[0])]:                
        if int(i) < (len(locals()['trainingFrames{}'.format("_" + file.split(".csv")[0])]))-3:
            j = int(i)+1
            k = int(i)+2
            
            combine = locals()['trainingFrames{}'.format("_" + file.split(".csv")[0])]['{}'.format(i)]
            combineTwo = locals()['trainingFrames{}'.format("_" + file.split(".csv")[0])]['{}'.format(j)]
            combineThree = locals()['trainingFrames{}'.format("_" + file.split(".csv")[0])]['{}'.format(k)]
                        
            combine = np.append(combine, combineTwo, axis=1)
            combine = np.append(combine, combineThree, axis=1)
            
            locals()['trainingCombo{}'.format("_" + file.split(".csv")[0])]['{}'.format(i)] = combine  

In [5]:
print(train)

      0           1           2
0     0  426.752398  275.903035
1     0  394.310455  277.394716
2     0  435.082628  331.251330
3     0  386.048717  333.176146
4     0  410.764179  369.639403
..   ..         ...         ...
967  80  424.101477  358.919397
968  80  380.217128  422.759701
969  80  414.270590  423.705627
970  80  394.806241  485.773869
971  80  415.025478  485.073556

[972 rows x 3 columns]


### Create a similar dataframe for the test videos
Does the same thing as above but for the test video names

In [6]:
# # open the .txt file which have names of test videos
# f = open("testlist_tennis.txt", "r")
# temp = f.read()
# videos = temp.split('\n')

# # creating a dataframe having video names
# test = pd.DataFrame()
# test['video_name'] = videos
# test

# Do the same for test videos

In [7]:
path="dataPoints_test/"

test = pd.DataFrame()

for filename in glob(os.path.join(path, '*.csv')):    
    file = filename.split("/")[1]
    test = pd.read_csv(path + file, header=None)
    locals()['testFrames{}'.format("_" + file.split(".csv")[0])] = {}
    locals()['testCombo{}'.format("_" + file.split(".csv")[0])] = {}
        
    size = len(convertFile[0]) // 12
    
    for i in range((len(test) + 12 - 1) // 12):
        data = test.to_numpy()[i * n:(i + 1) * n]
        data = np.delete(data, 0, 1)
        data = np.reshape(data, (24, 1))
        locals()['testFrames{}'.format("_" + file.split(".csv")[0])]['{}'.format(i)] = data

    for i in locals()['testFrames{}'.format("_" + file.split(".csv")[0])]:                
        if int(i) < len(locals()['testFrames{}'.format("_" + file.split(".csv")[0])])-3:
            j = int(i)+1
            k = int(i)+2
            
            combine = locals()['testFrames{}'.format("_" + file.split(".csv")[0])]['{}'.format(i)]
            combineTwo = locals()['testFrames{}'.format("_" + file.split(".csv")[0])]['{}'.format(j)]
            combineThree = locals()['testFrames{}'.format("_" + file.split(".csv")[0])]['{}'.format(k)]
            
            combine = np.append(combine, combineTwo, axis=1)
            combine = np.append(combine, combineThree, axis=1)
            
            locals()['testCombo{}'.format("_" + file.split(".csv")[0])]['{}'.format(i)] = combine            

In [8]:
print(test)

Empty DataFrame
Columns: []
Index: []


### Adding the tags
For both training and test sets: grabs the tag (0 or 1/good or bad serve) from the text file and stores in the respective dataframe.

In [9]:
# # creating tags for training videos
# train_video_tag = []
# for i in range(train.shape[0]):
#     train_video_tag.append(train['video_name'][i].split('_')[1].split('.')[0])
    
# train['tag'] = train_video_tag

# # creating tags for test videos
# test_video_tag = []
# for i in range(test.shape[0]):
#     test_video_tag.append(test['video_name'][i].split('_')[1].split('.')[0])
    
# test['tag'] = test_video_tag

In [10]:
#This is what the test dataframe looks like 
# test

In [11]:
# This is what the train dataframe looks like 
# train

### Frame Extraction and Storing
All frames are put in a folder named train_1

In [12]:
# # open the .txt file which have names of training videos
# f = open("trainlist_tennis.txt", "r")
# temp = f.read()
# videos = temp.split("\n")

# train = pd.DataFrame()
# train['video_name'] = videos

# # storing the frames from training videos
# for i in tqdm(range(len(file))):
#     count = 0
#     videoFile = train['video_name'][i]
#     cap = cv2.VideoCapture('myVids/'+videoFile)   # capturing the video from the given path
#     frameRate = cap.get(5) #frame rate
#     x=1
#     while(cap.isOpened()):
#         frameId = cap.get(1) #current frame number
#         ret, frame = cap.read()
#         if (ret != True):
#             break
#         if (frameId % math.floor(frameRate) == 0):
#             # storing the frames in a new folder named train_1
#             print("this is the split")
#             print(videoFile.split(" ")[0]) #since my videos aren't in separate folder, no need to split based on ("/")
            
#             #need to make sure that the train_1 has been created
#             filename ='train_1/' + videoFile.split(' ')[0] +"_frame%d.jpg" % count;count+=1
#             cv2.imwrite(filename, frame)
#     cap.release()

### .csv file
Get the names of all the frames with their corresponding tag and put the info into a .csv file

In [13]:
# #getting the names of all the images
# images = glob("train_1/*.jpg")
# train_image = []
# train_class = []

# for i in tqdm(range(len(images))):
#     # creating the image name
#     train_image.append(images[i].split("\\")[1])  #uncomment this for windows
#     #train_image.append(images[i].split("/")[1])  #uncomment this for mac
#     # creating the class of image
#     train_class.append(images[i].split("\\")[1].split(".")[0].split("_")[1]) #uncomment this for windows
#     #train_class.append(images[i].split("/")[1].split(".")[0].split("_")[1]) #uncomment this for mac
    
# # storing the images and their class in a dataframe
# train_data = pd.DataFrame()
# train_data['image'] = train_image
# train_data['class'] = train_class

# # converting the dataframe into csv file 
# train_data.to_csv('myVids/train_new.csv',header=True, index=False)

# Put all the training frames into a dummy thicc array and make them tags numerical

In [14]:
path="dataPoints_training/"

points = []
tags = []

for filename in glob(os.path.join(path, '*.csv')):    
    file = filename.split("/")[1]
    
    size = len(locals()['trainingCombo{}'.format("_" + file.split(".csv")[0])])
    
    for i in range(0, size-1):
        val = list(locals()['trainingCombo{}'.format("_" + file.split(".csv")[0])]["{}".format(i)])
        points.append(val)
    
    for i in range(0, size-1):
        tags.append(locals()['trainingCombo{}'.format("_" + file.split(".csv")[0])]["tag"])

X = np.array(points)

tags = pd.DataFrame(tags)


tags = tags.replace("b", 1)
tags = tags.replace("g", 0)

print(tags.loc[208][0])



0


# One hot encoding class

In [15]:
dummy_tags = to_categorical(tags)

# Split models

In [16]:
X_train, X_valid, y_train, y_valid = train_test_split(X, dummy_tags, test_size=0.2, random_state=42)    # preparing the validation set

In [22]:
X_train

array([[[389.94708228, 398.92796617, 403.82958794],
        [280.69094647, 277.66775881, 274.30778533],
        [408.75134895, 407.18433662, 417.40926898],
        ...,
        [541.89989854, 548.31110735, 542.343864  ],
        [409.91168189, 410.49700192, 405.53975977],
        [541.5077536 , 547.58752014, 540.39639128]],

       [[394.50415229, 512.72737377, 503.76598789],
        [243.79281961, 243.22488391, 243.49849716],
        [412.34244855, 494.64880799, 499.61417232],
        ...,
        [440.81366276, 478.24061056, 480.45150816],
        [440.2994475 , 459.23763527, 458.58786023],
        [444.90425318, 479.12217671, 481.17485789]],

       [[383.55831636, 370.29563815, 365.5964061 ],
        [297.97655395, 284.55326065, 269.59181017],
        [430.21933708, 434.49360295, 416.17863874],
        ...,
        [512.51496193, 500.72377721, 498.72272655],
        [391.23131422, 391.17388892, 403.53096691],
        [523.36482779, 510.69487932, 503.53163189]],

       ...,

      

In [23]:
X_train.shape

(167, 24, 3)

In [24]:
X_valid.shape

(42, 24, 3)

In [25]:
y_train.shape

(167, 2)

In [26]:
y_valid.shape

(42, 2)

# Shape into single dimension

In [27]:
X_train.shape, X_valid.shape
X_train = X_train.reshape(167, 24*3)
X_valid = X_valid.reshape(42, 24*3)

In [32]:
#normalize me 
train = X_train/X_train.max()      
X_valid = X_valid/X_train.max()

In [33]:
train

array([[0.69300758, 0.70896826, 0.71767935, ..., 0.96235873, 0.97316359,
        0.96038364],
       [0.70110633, 0.91121071, 0.89528468, ..., 0.79067657, 0.8514881 ,
        0.8551361 ],
       [0.68165357, 0.65808336, 0.64973196, ..., 0.93011542, 0.90759859,
        0.89486819],
       ...,
       [0.89865918, 0.89865918, 0.89865918, ..., 0.85883043, 0.85883043,
        0.85883043],
       [0.72207633, 0.72238604, 0.70866463, ..., 0.98336001, 0.98230341,
        0.96577889],
       [0.89429199, 0.88568946, 0.82206587, ..., 0.84690558, 0.86801431,
        0.88515428]])

# Build dat model

In [39]:
model = Sequential()
model.add(InputLayer((24*3,)))    # input layer
model.add(Dense(units=10, activation='sigmoid', input_shape=(24*3,))) # hidden layer
model.add(Dense(2, activation='softmax'))    # output layer

Instructions for updating:
Colocations handled automatically by placer.


In [40]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                730       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 22        
Total params: 752
Trainable params: 752
Non-trainable params: 0
_________________________________________________________________


# Weights file

In [41]:
# defining a function to save the weights of best model
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

# Compile model

In [42]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Create train dataframe

In [44]:
# path="dataPoints_training/"

# train = pd.DataFrame(columns=["file", "tag"])

# for filename in glob(os.path.join(path, '*.csv')):    
#     file = filename.split("/")[1]
    
#     size = len(locals()['trainingCombo{}'.format("_" + file.split(".csv")[0])])
    
#     category = locals()['trainingCombo{}'.format("_" + file.split(".csv")[0])]["tag"]
    
#     if category is "b":
#         category = 1
#     else:
#         category = 0
    
#     for i in range(0, size-1):
        
#         train = train.append({"file": file, "tag": category}, ignore_index=True)



# # for filename in glob(os.path.join(path, '*.csv')):    
# #     file = filename.split("/")[1]
# #     convertFile = pd.read_csv(path + file, header=None)
# #     locals()['trainingFrames{}'.format("_" + file.split(".csv")[0])] = {}
# #     locals()['trainingCombo{}'.format("_" + file.split(".csv")[0])] = {}
    
# #     tag = filename.split("/")[1].split("_")[3].split(".csv")[0]
# #     locals()['trainingCombo{}'.format("_" + file.split(".csv")[0])]["tag"] = tag            
        
# #     size = len(convertFile[0]) // 12
       
# #     n = 12
# #     for i in range((len(convertFile) + 12 - 1) // 12):
# #         data = convertFile.to_numpy()[i * n:(i + 1) * n]
# #         data = np.delete(data, 0, 1)
# #         data = np.reshape(data, (24, 1))
# #         locals()['trainingFrames{}'.format("_" + file.split(".csv")[0])]['{}'.format(i)] = data

# #     for i in locals()['trainingFrames{}'.format("_" + file.split(".csv")[0])]:                
# #         if int(i) < (len(locals()['trainingFrames{}'.format("_" + file.split(".csv")[0])]))-3:
# #             j = int(i)+1
# #             k = int(i)+2
            
# #             combine = locals()['trainingFrames{}'.format("_" + file.split(".csv")[0])]['{}'.format(i)]
# #             combineTwo = locals()['trainingFrames{}'.format("_" + file.split(".csv")[0])]['{}'.format(j)]
# #             combineThree = locals()['trainingFrames{}'.format("_" + file.split(".csv")[0])]['{}'.format(k)]
                        
# #             combine = np.append(combine, combineTwo, axis=1)
# #             combine = np.append(combine, combineThree, axis=1)
            
# #             locals()['trainingCombo{}'.format("_" + file.split(".csv")[0])]['{}'.format(i)] = combine         

        
# # print(trainingCombo{})

# Fit model

In [45]:
print("train", train, 'y-train', y_train)

train [[0.69300758 0.70896826 0.71767935 ... 0.96235873 0.97316359 0.96038364]
 [0.70110633 0.91121071 0.89528468 ... 0.79067657 0.8514881  0.8551361 ]
 [0.68165357 0.65808336 0.64973196 ... 0.93011542 0.90759859 0.89486819]
 ...
 [0.89865918 0.89865918 0.89865918 ... 0.85883043 0.85883043 0.85883043]
 [0.72207633 0.72238604 0.70866463 ... 0.98336001 0.98230341 0.96577889]
 [0.89429199 0.88568946 0.82206587 ... 0.84690558 0.86801431 0.88515428]] y-train [[1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [

In [46]:
# train['tag'].shape, y_train.shape

In [47]:
model.fit(train, y_train, epochs=10, validation_data=(X_valid, y_valid), callbacks=[mcp_save], batch_size=3)

Instructions for updating:
Use tf.cast instead.
Train on 167 samples, validate on 42 samples
Epoch 1/10
167/167 [==============================] - 0s 1ms/step - loss: 0.6191 - accuracy: 0.7006 - val_loss: 0.5518 - val_accuracy: 0.7619
Epoch 2/10
167/167 [==============================] - 0s 342us/step - loss: 0.6037 - accuracy: 0.7006 - val_loss: 0.5441 - val_accuracy: 0.7619
Epoch 3/10
167/167 [==============================] - 0s 448us/step - loss: 0.5963 - accuracy: 0.7006 - val_loss: 0.5401 - val_accuracy: 0.7619
Epoch 4/10
167/167 [==============================] - 0s 375us/step - loss: 0.5884 - accuracy: 0.7006 - val_loss: 0.5403 - val_accuracy: 0.7619
Epoch 5/10
167/167 [==============================] - 0s 499us/step - loss: 0.5815 - accuracy: 0.7006 - val_loss: 0.5238 - val_accuracy: 0.7619
Epoch 6/10
167/167 [==============================] - 0s 469us/step - loss: 0.5748 - accuracy: 0.7006 - val_loss: 0.5171 - val_accuracy: 0.7619
Epoch 7/10
167/167 [=========================

## What does this show us?

70% accuracy 

### Variable Observations:

In [34]:
train

array([[0.69300758, 0.70896826, 0.71767935, ..., 0.96235873, 0.97316359,
        0.96038364],
       [0.70110633, 0.91121071, 0.89528468, ..., 0.79067657, 0.8514881 ,
        0.8551361 ],
       [0.68165357, 0.65808336, 0.64973196, ..., 0.93011542, 0.90759859,
        0.89486819],
       ...,
       [0.89865918, 0.89865918, 0.89865918, ..., 0.85883043, 0.85883043,
        0.85883043],
       [0.72207633, 0.72238604, 0.70866463, ..., 0.98336001, 0.98230341,
        0.96577889],
       [0.89429199, 0.88568946, 0.82206587, ..., 0.84690558, 0.86801431,
        0.88515428]])

In [35]:
X_valid

array([[0.6271671 , 0.62914874, 0.6271671 , ..., 0.97510011, 0.95567618,
        0.97510011],
       [0.7006424 , 0.69972167, 0.70600887, ..., 0.99259487, 0.98871635,
        0.99228237],
       [0.58112463, 0.57956135, 0.57581185, ..., 0.86083634, 0.85716817,
        0.8524651 ],
       ...,
       [0.73831925, 0.73553202, 0.7249663 , ..., 0.98946526, 0.98931999,
        0.99282355],
       [0.66301755, 0.67288437, 0.67100741, ..., 0.95412762, 0.94933904,
        0.94534757],
       [0.72372745, 0.72282837, 0.7448155 , ..., 0.99307155, 0.99048894,
        0.98530218]])

In [37]:
y_train

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.

In [38]:
y_valid

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)